In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)
import numpy as np
import re
import spacy
import json
import gensim
from gensim import corpora
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
with open('Philips-juicer.json') as f:
  data = json.load(f)
reviews = data[0]['reviews']
df = pd.DataFrame(reviews)
df.head()

,Username,ReviewTitle,Rating,Verification,Date,Country,Review,Helpfull-Vote,no_of_Comments,imgs,video
0,AJ,Phlipls cheap qualityplastic and poor juicer,1,Verified Purchase,26 October 2018,India,The juicer is of very poor quality plastic and material. See pics attached of new one and a 1 year old one.Philips in ruining their name with such quality.,127,1,"[https://images-na.ssl-images-amazon.com/images/I/81TUeehxxGL._SY88.jpg, https://images-na.ssl-images-amazon.com/images/I/81vqJwKI9oL._SY88.jpg]",No Video
1,STRAIGHT TALK,Leaves too much juice in the discarded pulp,1,Verified Purchase,21 September 2018,India,It leaves too much juice in the discarded pulp and therefore a complete failure for my purpose.I returned the juicer after using it once.It is not the quality I expected.It would be misleading to ...,75,2,No Images,No Video
2,Indravadan Damor,Nice juicer,5,Verified Purchase,3 April 2018,India,"I am writing a review here after using it almost 2 year. I really like this juicer to use it in my routine life. By using this juicer I make juice of carrot, orange, pineapole, bottle guard and m...",65,1,No Images,No Video
3,Abhinav P.,Powerful and Durable Juicer,4,Verified Purchase,23 December 2018,India,"Writing after 1 month of use, Pros and cons :- Carrot and beetroot juice pic is attached.Pro's :-Juicer is super powerful.Build quality is good.Easy to use and cleanNo juice left in Vegetable resi...",33,0,[https://images-na.ssl-images-amazon.com/images/I/71gLe2s5YfL._SY88.jpg],No Video
4,Sandeep K.,Perfect juicer ...,5,Verified Purchase,6 April 2018,India,This is your juicing solution ...it juices everything even leafy vegetables ...u can put whole fruits and the pulp is dry with absolutely no juice left ...I am loving the product ...it makes good ...,24,0,No Images,No Video


In [ ]:
df.shape

(1627, 11)

# Complete_Data

In [ ]:
reviews_col = df['ReviewTitle'] + " " + df['Review']
reviews = pd.DataFrame(reviews_col,df.index,['reviews'])
reviews.head()

,reviews
0,Phlipls cheap qualityplastic and poor juicer The juicer is of very poor quality plastic and material. See pics attached of new one and a 1 year old one.Philips in ruining their name with such qual...
1,Leaves too much juice in the discarded pulp It leaves too much juice in the discarded pulp and therefore a complete failure for my purpose.I returned the juicer after using it once.It is not the q...
2,"Nice juicer I am writing a review here after using it almost 2 year. I really like this juicer to use it in my routine life. By using this juicer I make juice of carrot, orange, pineapole, bottle..."
3,"Powerful and Durable Juicer Writing after 1 month of use, Pros and cons :- Carrot and beetroot juice pic is attached.Pro's :-Juicer is super powerful.Build quality is good.Easy to use and cleanNo ..."
4,Perfect juicer ... This is your juicing solution ...it juices everything even leafy vegetables ...u can put whole fruits and the pulp is dry with absolutely no juice left ...I am loving the produc...


In [ ]:
reviews['review_processed'] = reviews['reviews'].map(lambda x: re.sub('[,\.!?]','',x))
reviews['review_processed'] = reviews['review_processed'].map(lambda x:x.lower())

<input>:1: DeprecationWarning: invalid escape sequence \.
<input>:1: DeprecationWarning: invalid escape sequence \.
<input>:1: DeprecationWarning: invalid escape sequence \.
<input>:1: DeprecationWarning: invalid escape sequence \.
<input>:1: DeprecationWarning: invalid escape sequence \.
<input>:1: DeprecationWarning: invalid escape sequence \.
<ipython-input-119-c4b5c66fa377>:1: DeprecationWarning: invalid escape sequence \.
  reviews['review_processed'] = reviews['reviews'].map(lambda x: re.sub('[,\.!?]','',x))


In [ ]:
reviews.head()

,reviews,review_processed
0,Phlipls cheap qualityplastic and poor juicer The juicer is of very poor quality plastic and material. See pics attached of new one and a 1 year old one.Philips in ruining their name with such qual...,phlipls cheap qualityplastic and poor juicer the juicer is of very poor quality plastic and material see pics attached of new one and a 1 year old onephilips in ruining their name with such quality
1,Leaves too much juice in the discarded pulp It leaves too much juice in the discarded pulp and therefore a complete failure for my purpose.I returned the juicer after using it once.It is not the q...,leaves too much juice in the discarded pulp it leaves too much juice in the discarded pulp and therefore a complete failure for my purposei returned the juicer after using it onceit is not the qua...
2,"Nice juicer I am writing a review here after using it almost 2 year. I really like this juicer to use it in my routine life. By using this juicer I make juice of carrot, orange, pineapole, bottle...",nice juicer i am writing a review here after using it almost 2 year i really like this juicer to use it in my routine life by using this juicer i make juice of carrot orange pineapole bottle guar...
3,"Powerful and Durable Juicer Writing after 1 month of use, Pros and cons :- Carrot and beetroot juice pic is attached.Pro's :-Juicer is super powerful.Build quality is good.Easy to use and cleanNo ...",powerful and durable juicer writing after 1 month of use pros and cons :- carrot and beetroot juice pic is attachedpro's :-juicer is super powerfulbuild quality is goodeasy to use and cleanno juic...
4,Perfect juicer ... This is your juicing solution ...it juices everything even leafy vegetables ...u can put whole fruits and the pulp is dry with absolutely no juice left ...I am loving the produc...,perfect juicer this is your juicing solution it juices everything even leafy vegetables u can put whole fruits and the pulp is dry with absolutely no juice left i am loving the product it makes g...


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True))

data = reviews.review_processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[0][:])

['phlipls', 'cheap', 'qualityplastic', 'and', 'poor', 'juicer', 'the', 'juicer', 'is', 'of', 'very', 'poor', 'quality', 'plastic', 'and', 'material', 'see', 'pics', 'attached', 'of', 'new', 'one', 'and', 'year', 'old', 'onephilips', 'in', 'ruining', 'their', 'name', 'with', 'such', 'quality']


In [ ]:
bigram = gensim.models.Phrases(reviews.review_processed, min_count=5, threshold=50)
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess


stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words)

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

['phlipls', 'cheap', 'qualityplastic', 'poor', 'juicer', 'juicer', 'very', 'poor', 'quality', 'plastic', 'material', 'see', 'pic', 'attach', 'new', 'year', 'old', 'onephilip', 'ruin', 'name', 'such', 'quality']


In [ ]:
print(data_lemmatized[0])

['phlipls', 'cheap', 'qualityplastic', 'poor', 'juicer', 'juicer', 'very', 'poor', 'quality', 'plastic', 'material', 'see', 'pic', 'attach', 'new', 'year', 'old', 'onephilip', 'ruin', 'name', 'such', 'quality']


In [ ]:
id2word = corpora.Dictionary(data_lemmatized)

texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=4, 
                                       random_state=14,
                                       passes=20)

In [ ]:
from pprint import pprint

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.074*"product" + 0.073*"good" + 0.045*"easy" + 0.045*"very" + 0.038*"star" '
  '+ 0.036*"juicer" + 0.029*"clean" + 0.027*"use" + 0.024*"juice" + '
  '0.020*"excellent"'),
 (1,
  '0.054*"juice" + 0.029*"juicer" + 0.024*"good" + 0.020*"fruit" + '
  '0.015*"pulp" + 0.014*"can" + 0.011*"extract" + 0.010*"use" + 0.010*"make" + '
  '0.010*"clean"'),
 (2,
  '0.030*"product" + 0.022*"juicer" + 0.020*"use" + 0.015*"juice" + '
  '0.014*"very" + 0.013*"get" + 0.013*"buy" + 0.012*"work" + 0.011*"day" + '
  '0.010*"bad"'),
 (3,
  '0.033*"product" + 0.012*"receive" + 0.011*"poor" + 0.010*"damage" + '
  '0.010*"work" + 0.009*"juicer" + 0.009*"well" + 0.008*"purchase" + '
  '0.007*"open" + 0.007*"damaged"')]


This is meaningful.


*   Topic 1 is about good juicer and cleanliness
*   Topic 2 is about vegeatable, fruits, pulp etc
*   Topic 3 is about not working and returns and service
*   Topic 4 is about recived damaged product etc

This can also be seen from the visualization below










In [ ]:
import pyLDAvis.gensim
import pyLDAvis

pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.153592 -0.091023       1        1  34.376217
1      0.055771  0.108262       2        1  30.661285
2     -0.048140  0.045659       3        1  28.231037
3     -0.161224 -0.062898       4        1   6.731461, topic_info=          Term        Freq       Total Category  logprob  loglift
97     product  952.000000  952.000000  Default  30.0000  30.0000
934       star  336.000000  336.000000  Default  29.0000  29.0000
54        easy  430.000000  430.000000  Default  28.0000  28.0000
56        good  849.000000  849.000000  Default  27.0000  27.0000
520  excellent  168.000000  168.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
260        buy    9.943456  218.012085   Topic4  -5.1122  -0.3893
220      motor    7.085412   81.151395   Topic4  -5.4511   0.2601
345       just    7.618515  125.379959   Topic4  -5.3785  -0.1024
17        very    6.961579  537.610777   Topic4  -5.4687  -1.6484
349       look    6.352692   83.766002   Topic4  -5.5602   0.1192

[291 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
482       3  0.243372  additional
482       4  0.730115  additional
132       1  0.070152         all
132       2  0.245533         all
132       3  0.350761         all
...     ...       ...         ...
255       4  0.052262       worth
70        3  0.928473       write
18        1  0.027969        year
18        3  0.755166        year
18        4  0.195784        year

[512 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4])

# Modelling based on product rating



In [ ]:
reviews['rating'] = df.Rating
reviews.head()

,reviews,review_processed,rating
0,Phlipls cheap qualityplastic and poor juicer The juicer is of very poor quality plastic and material. See pics attached of new one and a 1 year old one.Philips in ruining their name with such qual...,phlipls cheap qualityplastic and poor juicer the juicer is of very poor quality plastic and material see pics attached of new one and a 1 year old onephilips in ruining their name with such quality,1
1,Leaves too much juice in the discarded pulp It leaves too much juice in the discarded pulp and therefore a complete failure for my purpose.I returned the juicer after using it once.It is not the q...,leaves too much juice in the discarded pulp it leaves too much juice in the discarded pulp and therefore a complete failure for my purposei returned the juicer after using it onceit is not the qua...,1
2,"Nice juicer I am writing a review here after using it almost 2 year. I really like this juicer to use it in my routine life. By using this juicer I make juice of carrot, orange, pineapole, bottle...",nice juicer i am writing a review here after using it almost 2 year i really like this juicer to use it in my routine life by using this juicer i make juice of carrot orange pineapole bottle guar...,5
3,"Powerful and Durable Juicer Writing after 1 month of use, Pros and cons :- Carrot and beetroot juice pic is attached.Pro's :-Juicer is super powerful.Build quality is good.Easy to use and cleanNo ...",powerful and durable juicer writing after 1 month of use pros and cons :- carrot and beetroot juice pic is attachedpro's :-juicer is super powerfulbuild quality is goodeasy to use and cleanno juic...,4
4,Perfect juicer ... This is your juicing solution ...it juices everything even leafy vegetables ...u can put whole fruits and the pulp is dry with absolutely no juice left ...I am loving the produc...,perfect juicer this is your juicing solution it juices everything even leafy vegetables u can put whole fruits and the pulp is dry with absolutely no juice left i am loving the product it makes g...,5


In [ ]:
reviews.rating.value_counts()

5    919
4    321
1    233
3    101
2     53
Name: rating, dtype: int64

This can be divided into good and bad reviews based on rating such that good reviews = rating(4,5) and bad reviews = rating(1,2,3)

In [ ]:
mapper = {1:'bad',2:'bad',3:'bad',4:'good',5:'good'}
reviews['rating'] = reviews['rating'].map(mapper)
reviews.head()

,reviews,review_processed,rating
0,Phlipls cheap qualityplastic and poor juicer The juicer is of very poor quality plastic and material. See pics attached of new one and a 1 year old one.Philips in ruining their name with such qual...,phlipls cheap qualityplastic and poor juicer the juicer is of very poor quality plastic and material see pics attached of new one and a 1 year old onephilips in ruining their name with such quality,bad
1,Leaves too much juice in the discarded pulp It leaves too much juice in the discarded pulp and therefore a complete failure for my purpose.I returned the juicer after using it once.It is not the q...,leaves too much juice in the discarded pulp it leaves too much juice in the discarded pulp and therefore a complete failure for my purposei returned the juicer after using it onceit is not the qua...,bad
2,"Nice juicer I am writing a review here after using it almost 2 year. I really like this juicer to use it in my routine life. By using this juicer I make juice of carrot, orange, pineapole, bottle...",nice juicer i am writing a review here after using it almost 2 year i really like this juicer to use it in my routine life by using this juicer i make juice of carrot orange pineapole bottle guar...,good
3,"Powerful and Durable Juicer Writing after 1 month of use, Pros and cons :- Carrot and beetroot juice pic is attached.Pro's :-Juicer is super powerful.Build quality is good.Easy to use and cleanNo ...",powerful and durable juicer writing after 1 month of use pros and cons :- carrot and beetroot juice pic is attachedpro's :-juicer is super powerfulbuild quality is goodeasy to use and cleanno juic...,good
4,Perfect juicer ... This is your juicing solution ...it juices everything even leafy vegetables ...u can put whole fruits and the pulp is dry with absolutely no juice left ...I am loving the produc...,perfect juicer this is your juicing solution it juices everything even leafy vegetables u can put whole fruits and the pulp is dry with absolutely no juice left i am loving the product it makes g...,good


## Good reviews

In [ ]:
df_good = reviews[reviews['rating'] == 'good']['review_processed']
df_good.head()

2     nice juicer i am writing a review here after using it almost 2 year i really like this juicer to use it in my routine life by using this juicer i make juice of carrot  orange pineapole bottle guar...
3     powerful and durable juicer writing after 1 month of use pros and cons :- carrot and beetroot juice pic is attachedpro's :-juicer is super powerfulbuild quality is goodeasy to use and cleanno juic...
4     perfect juicer  this is your juicing solution it juices everything even leafy vegetables u can put whole fruits and the pulp is dry with absolutely no juice left i am loving the product it makes g...
6     hr 1863 is the best juicer for the range ₹ 5000-10000 this is an outstanding juicer from philips its centrifugal technique makes it different from other juicers extracting juice from fruits like g...
12    nice juicer i am writing a review here after using it almost 2 year i really like this juicer to use it in my routine life by using this juicer i make juice of carrot  or

In [ ]:
good_data = df_good.tolist()
good_data_words = list(sent_to_words(good_data))
print(good_data_words[0][:])

['nice', 'juicer', 'am', 'writing', 'review', 'here', 'after', 'using', 'it', 'almost', 'year', 'really', 'like', 'this', 'juicer', 'to', 'use', 'it', 'in', 'my', 'routine', 'life', 'by', 'using', 'this', 'juicer', 'make', 'juice', 'of', 'carrot', 'orange', 'pineapole', 'bottle', 'guard', 'and', 'more', 'it', 'is', 'very', 'easy', 'and', 'fast', 'the', 'only', 'one', 'thing', 'don', 'like', 'about', 'this', 'juicer', 'is', 'it', 'makes', 'too', 'much', 'noise', 'when', 'juicing', 'beetroot', 'but', 'still', 'it', 'is', 'good', 'choice', 'in', 'compare', 'to', 'others']


In [ ]:
bigram = gensim.models.Phrases(df_good, min_count=5, threshold=50)
trigram = gensim.models.Phrases(bigram[good_data_words], threshold=50)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
good_data_words_nostops = remove_stopwords(good_data_words)

good_data_words_bigrams = make_bigrams(good_data_words)

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

good_data_lemmatized = lemmatization(good_data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
print(good_data_lemmatized[0])

['nice', 'write', 'review', 'here', 'use', 'almost', 'year', 'really', 'juicer', 'use', 'routine', 'life', 'use', 'juicer', 'make', 'juice', 'more', 'very', 'easy', 'fast', 'only', 'thing', 'juicer', 'make', 'too', 'much', 'noise', 'when', 'juice', 'beetroot', 'still', 'good', 'choice', 'compare', 'other']


In [ ]:
id2word = corpora.Dictionary(good_data_lemmatized)

texts = good_data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 3), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 3), (25, 1), (26, 1), (27, 1), (28, 1)]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=2, 
                                       random_state=14,
                                       passes=20)

In [ ]:
from pprint import pprint

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.048*"star" + 0.030*"juice" + 0.024*"juicer" + 0.015*"fruit" + '
  '0.014*"good" + 0.012*"pulp" + 0.011*"excellent" + 0.010*"can" + '
  '0.010*"product" + 0.010*"more"'),
 (1,
  '0.059*"good" + 0.057*"product" + 0.036*"juicer" + 0.036*"very" + '
  '0.035*"easy" + 0.031*"juice" + 0.027*"use" + 0.022*"clean" + 0.012*"great" '
  '+ 0.011*"buy"')]


This result captures the qualities of a good review. It can be seen that

*   Topic 1 is about the juicer being able to handle fruits and pulp really well.
*   Topic 2 is about the cleanliness,value for money and being easy to use


In [ ]:
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

PreparedData(topic_coordinates=             x    y  topics  cluster       Freq
topic                                          
1      0.07848  0.0       1        1  65.995202
0     -0.07848  0.0       2        1  34.004798, topic_info=        Term        Freq       Total Category  logprob  loglift
636     star  284.000000  284.000000  Default  30.0000  30.0000
4       easy  400.000000  400.000000  Default  29.0000  29.0000
79      pulp   73.000000   73.000000  Default  28.0000  28.0000
57   product  694.000000  694.000000  Default  27.0000  27.0000
25      very  431.000000  431.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
128      buy   28.176700  146.571379   Topic2  -5.3160  -0.5703
40   extract   23.881491   78.072891   Topic2  -5.4814  -0.1059
25      very   34.226729  431.371812   Topic2  -5.1215  -1.4553
71     clean   30.241220  272.958473   Topic2  -5.2453  -1.1214
334       go   22.079623   64.739774   Topic2  -5.5598   0.0029

[164 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
266       1  0.967597   amazing
266       2  0.031213   amazing
164       1  0.952537  assemble
164       2  0.057152  assemble
1120      1  0.162115   atleast
...     ...       ...       ...
376       2  0.139175      work
162       1  0.780563     worth
162       2  0.217534     worth
28        1  0.955146      year
28        2  0.056185      year

[211 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

## Bad reviews

In [ ]:
df_bad = reviews[reviews['rating'] == 'bad']['review_processed']
df_bad.head()

0      phlipls cheap qualityplastic and poor juicer the juicer is of very poor quality plastic and material see pics attached of new one and a 1 year old onephilips in ruining their name with such quality
1    leaves too much juice in the discarded pulp it leaves too much juice in the discarded pulp and therefore a complete failure for my purposei returned the juicer after using it onceit is not the qua...
5    my machine's lower part got stuck after only 10 uses  my machine's lower part got stuck after only 10 uses  it is not coming out there is no response from the customer service department have call...
7    very disappointing we purchased in a hope it'd be better than others first two days it worked amazingbut on third day its lower part got stuck n glued on its own we had to call the technician to g...
8    the motor is heating very fast we have been using the juicer for hardly 2 minutes to take out kakadi (vegetable) juice for daily consumption but on 3 ocassions due to motor he

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True))

bad_data = df_bad.tolist()
bad_data_words = list(sent_to_words(bad_data))
print(bad_data_words[0][:])

['phlipls', 'cheap', 'qualityplastic', 'and', 'poor', 'juicer', 'the', 'juicer', 'is', 'of', 'very', 'poor', 'quality', 'plastic', 'and', 'material', 'see', 'pics', 'attached', 'of', 'new', 'one', 'and', 'year', 'old', 'onephilips', 'in', 'ruining', 'their', 'name', 'with', 'such', 'quality']


In [ ]:
bigram = gensim.models.Phrases(df_bad, min_count=5, threshold=50)
trigram = gensim.models.Phrases(bigram[bad_data_words], threshold=50)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
bad_data_words_nostops = remove_stopwords(bad_data_words)

bad_data_words_bigrams = make_bigrams(bad_data_words)

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

bad_data_lemmatized = lemmatization(bad_data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
print(bad_data_lemmatized[0])

['phlipls', 'cheap', 'qualityplastic', 'poor', 'juicer', 'juicer', 'very', 'poor', 'quality', 'plastic', 'material', 'see', 'pic', 'attach', 'new', 'year', 'old', 'onephilip', 'ruin', 'name', 'such', 'quality']


In [ ]:
id2word = corpora.Dictionary(bad_data_lemmatized)

texts = bad_data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=2, 
                                       random_state=14,
                                       passes=20)

In [ ]:
from pprint import pprint

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.048*"product" + 0.024*"work" + 0.013*"service" + 0.013*"good" + '
  '0.012*"buy" + 0.012*"day" + 0.011*"bad" + 0.011*"star" + 0.010*"get" + '
  '0.010*"use"'),
 (1,
  '0.030*"juice" + 0.025*"product" + 0.021*"juicer" + 0.016*"very" + '
  '0.014*"use" + 0.014*"good" + 0.011*"pulp" + 0.010*"clean" + 0.010*"waste" + '
  '0.008*"quality"')]


From modelling the bad reviews, its evident that

*   Topic 1 is about it not working and poor sevice.
*   Topic 2 is about the build quality and defective.

In [ ]:
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

PreparedData(topic_coordinates=              x    y  topics  cluster      Freq
topic                                          
1      0.079077  0.0       1        1  64.53444
0     -0.079077  0.0       2        1  35.46556, topic_info=        Term        Freq       Total Category  logprob  loglift
86   product  252.000000  252.000000  Default  30.0000  30.0000
33     juice  153.000000  153.000000  Default  29.0000  29.0000
100     work   83.000000   83.000000  Default  28.0000  28.0000
301     stop   26.000000   26.000000  Default  27.0000  27.0000
39      pulp   56.000000   56.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
229     will   15.793457   36.692827   Topic2  -5.1473   0.1936
122    motor   19.469774   57.766224   Topic2  -4.9380  -0.0509
2     juicer   25.509200  128.841227   Topic2  -4.6679  -0.5829
17      very   22.800403  103.480748   Topic2  -4.7801  -0.4760
42    return   15.195620   48.606867   Topic2  -5.1859  -0.1261

[150 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
63        1  0.911489           also
63        2  0.098539           also
336       2  0.874183            ask
133       1  0.927577       assemble
133       2  0.042163       assemble
...     ...       ...            ...
100       1  0.226561           work
100       2  0.775078           work
1152      2  0.915851  workingplease
164       1  0.384622          worth
164       2  0.604407          worth

[186 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

# Result

Combining the results from modelling of good and bad reviews, we can extract features like:


*   Fruits and Pulp
*   Clean
*   Easy to Use
*   Quality
*   Value for Money
*   Not Working
*   Good Juicer
*   Damaged Product
*   Service Center


